In [237]:
import pandas as pd

In [238]:
df = pd.read_csv('tmdb_movies.csv')

In [239]:
df.head(7)

,id,title,release_date,vote_average,overview,poster_path,original_language,genre_names,popularity,vote_count,...,runtime,spoken_languages,budget,revenue,status,tagline,homepage,imdb_id,cast,crew
0,278,The Shawshank Redemption,1994-09-23,8.700,Imprisoned in the 1940s for the double murder ...,https://image.tmdb.org/t/p/w500/9cqNxx0GxF0bfl...,en,"Drama, Crime",157.766,27254,...,142,English,25000000,28341469,Released,Fear can hold you prisoner. Hope can set you f...,NaN,tt0111161,"Tim Robbins, Morgan Freeman, Bob Gunton",Frank Darabont
1,238,The Godfather,1972-03-14,8.689,"Spanning the years 1945 to 1955, a chronicle o...",https://image.tmdb.org/t/p/w500/3bhkrj58Vtu7en...,en,"Drama, Crime",190.434,20699,...,175,"English, Italiano, Latin",6000000,245066411,Released,An offer you can't refuse.,http://www.thegodfather.com/,tt0068646,"Marlon Brando, Al Pacino, James Caan",Francis Ford Coppola
2,240,The Godfather Part II,1974-12-20,8.600,In the continuing saga of the Corleone crime f...,https://image.tmdb.org/t/p/w500/hek3koDUyRQk7F...,en,"Drama, Crime",96.518,12483,...,202,"English, Italiano, Latin, EspaÃ±ol",13000000,102600000,Released,The rise and fall of the Corleone empire.,NaN,tt0071562,"Al Pacino, Robert Duvall, Diane Keaton",Francis Ford Coppola
3,424,Schindler's List,1993-12-15,8.567,The true story of how businessman Oskar Schind...,https://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUY...,en,"Drama, History, War",68.609,15900,...,195,"Deutsch, Polski, ×¢Ö´×‘Ö°×¨Ö´×™×ª, English",22000000,321365567,Released,"Whoever saves one life, saves the world entire.",http://www.schindlerslist.com/,tt0108052,"Liam Neeson, Ben Kingsley, Ralph Fiennes",Steven Spielberg
4,389,12 Angry Men,1957-04-10,8.500,The defense and the prosecution have rested an...,https://image.tmdb.org/t/p/w500/ow3wq89wM8qd5X...,en,Drama,48.220,8697,...,97,English,397751,4360000,Released,Life is in their hands â€” Death is on their m...,NaN,tt0050083,"Martin Balsam, John Fiedler, Lee J. Cobb",Sidney Lumet
5,129,Spirited Away,2001-07-20,8.500,"A young girl, Chihiro, becomes trapped in a st...",https://image.tmdb.org/t/p/w500/39wmItIWsg5sZM...,ja,"Animation, Family, Fantasy",90.456,16561,...,125,æ—¥æœ¬èªž,19000000,274925095,Released,NaN,http://movies.disney.com/spirited-away,tt0245429,"Rumi Hiiragi, Miyu Irino, Mari Natsuki",Hayao Miyazaki
6,19404,Dilwale Dulhania Le Jayenge,1995-10-20,8.500,"Raj is a rich, carefree, happy-go-lucky second...",https://image.tmdb.org/t/p/w500/lfRkUr7DYdHldA...,hi,"Comedy, Drama, Romance",40.315,4439,...,190,à¤¹à¤¿à¤¨à¥à¤¦à¥€,13200000,100000000,Released,"Comeâ€¦ Fall In love, All Over Againâ€¦",NaN,tt0112870,"Kajol, Shah Rukh Khan, Amrish Puri",Aditya Chopra


In [240]:
df.columns

Index(['id', 'title', 'release_date', 'vote_average', 'overview',
       'poster_path', 'original_language', 'genre_names', 'popularity',
       'vote_count', 'backdrop_path', 'original_title', 'adult', 'video',
       'production_companies', 'production_countries', 'runtime',
       'spoken_languages', 'budget', 'revenue', 'status', 'tagline',
       'homepage', 'imdb_id', 'cast', 'crew'],
      dtype='object')

In [241]:
df['imdb_id'].isna().sum()

6

In [242]:
df = df.dropna(subset=['imdb_id'])


In [243]:
df.shape

(9789, 26)

In [244]:
df = df.drop(columns=['homepage'])

In [245]:
df.columns

Index(['id', 'title', 'release_date', 'vote_average', 'overview',
       'poster_path', 'original_language', 'genre_names', 'popularity',
       'vote_count', 'backdrop_path', 'original_title', 'adult', 'video',
       'production_companies', 'production_countries', 'runtime',
       'spoken_languages', 'budget', 'revenue', 'status', 'tagline', 'imdb_id',
       'cast', 'crew'],
      dtype='object')

In [246]:
df['crew'] = df['crew'].fillna("Unknown")

In [247]:
df['tagline'].isna().sum()

1535

In [248]:
print(df.loc[1, 'tagline'])

An offer you can't refuse.


In [249]:
nan_indices = df[df['tagline'].isna()].index


In [250]:
nan_indices[3]

30

In [251]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

df_filled = df[df['tagline'].notna()].copy()
df_missing = df[df['tagline'].isna()].copy()

df_filled['combined_text'] = (
    df_filled['title'].fillna('') + " " +
    df_filled['overview'].fillna('') + " " +
    df_filled['genre_names'].fillna('')
)

df_missing['combined_text'] = (
    df_missing['title'].fillna('') + " " +
    df_missing['overview'].fillna('') + " " +
    df_missing['genre_names'].fillna('')
)

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_filled['combined_text'])

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(tfidf_matrix)

tfidf_missing = tfidf.transform(df_missing['combined_text'])

distances, indices = knn.kneighbors(tfidf_missing)

for i, idx_list in enumerate(indices):
    for idx in idx_list:
        # Проверяем, что у соседа есть 'tagline'
        if pd.notna(df_filled.iloc[idx]['tagline']):
            df_missing.iloc[i, df_missing.columns.get_loc('tagline')] = df_filled.iloc[idx]['tagline']
            break

df = pd.concat([df_filled, df_missing])

print("Taglines updated using KNN and saved successfully!")

Taglines updated using KNN and saved successfully!


In [252]:
print(df_updated.loc[30, 'tagline'])

Far From Grace


In [253]:
df = df.drop_duplicates(subset=['imdb_id'], keep='first')

In [254]:
df.shape

(9787, 26)

In [257]:
df.isna().sum()

id                       0
title                    0
release_date             0
vote_average             0
overview                 0
poster_path              1
original_language        0
genre_names              1
popularity               0
vote_count               0
backdrop_path            4
original_title           0
adult                    0
video                    0
production_companies    31
production_countries     5
runtime                  0
spoken_languages         4
budget                   0
revenue                  0
status                   0
tagline                  0
imdb_id                  0
cast                    25
crew                     0
combined_text            0
dtype: int64

In [258]:
df = df.dropna()

In [260]:
df.isna().sum()

id                      0
title                   0
release_date            0
vote_average            0
overview                0
poster_path             0
original_language       0
genre_names             0
popularity              0
vote_count              0
backdrop_path           0
original_title          0
adult                   0
video                   0
production_companies    0
production_countries    0
runtime                 0
spoken_languages        0
budget                  0
revenue                 0
status                  0
tagline                 0
imdb_id                 0
cast                    0
crew                    0
combined_text           0
dtype: int64

In [261]:
df.to_csv('clean_tmdb_movies.csv')